# Data loading
We start again as usuall by loading all the data from the successful and from the unsuccessful companies.

1. We import relevant libraries

In [285]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from functools import reduce
import math
import urllib
import difflib

Then we load the data:
1. load the excel files.

In [286]:
'''
First we load both spreadsheets and extra all the names of the workbooks
'''

xls_Success_Comp       = pd.ExcelFile('data/success/Moneyball 1.1_ Success.xlsx')
xls_Success_Comp_Names = xls_Success_Comp.sheet_names


xls_Unsuccess_Comp = pd.ExcelFile('data/failure/Moneyball 1.1_ Fail.xlsx')
xls_Unsuccess_Comp_Sheet_Names = xls_Unsuccess_Comp.sheet_names


2. Load each sheet into a dataframe.

In [287]:
'''
Importing all of the excel sheets of successful informaiton into a list of dataframes
'''
df_list_SuccessComp = [pd.read_excel(r'data/success/Moneyball 1.1_ Success.xlsx',
                                         sheet_name= sheetname) for sheetname in xls_Success_Comp_Names]
                                         
# df_SuccessfulCompanies[1].drop(index = [503, 504], inplace=True)
# df_SuccessfulCompanies[2].drop(index = [573, 574], inplace=True)
# df_SuccessfulCompanies[3].drop(index = [522, 523], inplace=True)

'''
Importing all of the excel sheets of unsuccessful informaiton, into a list of dataframes

'''
df_list_UnsuccessComp = [pd.read_excel(r'data/failure/Moneyball 1.1_ Fail.xlsx',
                                         sheet_name= sheetname) for sheetname in xls_Unsuccess_Comp_Sheet_Names]


In [288]:
# lets do some sanity checks
print(len(df_list_SuccessComp))

# df_list_SuccessfulCompanies[0].head()
df_list_SuccessComp[3].head()
df_list_SuccessComp[5].head()


6


,README
0,Using the 'investors_uuids' in the 'Funding Ro...
1,https://drive.google.com/file/d/1wcMns7E_cAD0R...


We can see that the Excel spreadsheet contains two sheets with only links to the CSV files of founder linkedin profiles and investor profiles, we have downloaded those and to get started we will not concern ourselves with the data from these two spreadsheets. I will drop them for now.

In [289]:
indexes = [3,4,5]
for index in sorted(indexes, reverse=True):
    del df_list_SuccessComp[index]
    del df_list_UnsuccessComp[index]

len(df_list_SuccessComp), len(df_list_UnsuccessComp)

(3, 3)

We find that the same org_uuid appear several times in the funding rounds sheet.In addition we find that there is a slight discrepancy between the number of org_uuid's between the founding rounds and the company dataframe. Therefore, we will create unique new columns for each org_uuid for the founding rounds dataframe to describe the amount of founding raised, the date, and the amount of founding raised.

In [211]:
# print the unique values of the relevant columns to get an overview
for col in list(df_list_SuccessComp[3])[2:]:
    print(df_list_SuccessComp[3][col].unique())

['seed' 'series_a' 'angel' 'pre_seed']
['2022-03-22 22:57:00.000000 UTC' '2019-01-02 03:34:58.000000 UTC'
 '2022-03-15 20:15:27.000000 UTC' ... '2018-12-22 10:45:38.000000 UTC'
 '2022-05-06 21:50:48.000000 UTC' '2015-07-30 09:26:41.000000 UTC']
[4.500e+08 1.825e+08 2.000e+08 ... 2.080e+07 6.600e+07 6.600e+06]
[39. 15. 20. 17. 30. 18. 36. 25. 40. 22. 29. 37. 42. 68. 24. 27. 41. 19.
 nan 21. 84. 38.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.
 16.]
['Adidas,Animoca Brands,Artist Capital Management,Atlas Benjelloun,BOND,Calvin Broadus,Cobalt Capital,Coinbase,Colin Kaepernick,DIGITAL,Dez Bryant,Eden Block,FTX Ventures,Friendly Trading 2,GV,Genies,Hashed,J.A.S Ventures,Justin David Blau,Lightspeed Venture Partners,LionTree,Mark Cuban,MoonPay,SV Angel,Samsung NEXT,Sergio Giavanni Kitchens,Seven Seven Six,Sound Ventures,Standard Crypto,Steve Aoki,Tess Ventures,The Forest Road Company,The Sandbox,Thrive Capital,Tiger Global Management,Time Ventures,Timothy Mosley,VaynerFund,a16z cr

Now we create a new column for each ['seed' 'series_a' 'angel' 'pre_seed'] the date and the round_created_at, raised_amount_usd, investor_count, investors_names, investors_uuids

In [212]:
# create the columns
invest_type_array = df_list_SuccessComp[3]['investment_type'].unique()
relevant_col_list = list(df_list_SuccessComp[3])[3:]
for round in invest_type_array:
    for col in relevant_col_list:
        df_list_SuccessComp[3][round + '_' + col] = ""
        df_list_UnsuccessComp[3][round + '_' + col] = ""


df_list_SuccessComp[3].head()

,org_uuid,org_name,investment_type,round_created_at,raised_amount_usd,investor_count,investors_names,investors_uuids,seed_round_created_at,seed_raised_amount_usd,...,angel_round_created_at,angel_raised_amount_usd,angel_investor_count,angel_investors_names,angel_investors_uuids,pre_seed_round_created_at,pre_seed_raised_amount_usd,pre_seed_investor_count,pre_seed_investors_names,pre_seed_investors_uuids
0,b42a3bec-1f35-42a3-9d5a-e6d3199440c6,Yuga Labs,seed,2022-03-22 22:57:00.000000 UTC,450000000.0,39.0,"Adidas,Animoca Brands,Artist Capital Managemen...","eec354de-1ed2-254d-647b-f218643791d9,8f1b5d66-...",,,...,,,,,,,,,,
1,8335acfe-629e-4073-b857-205b3b5d2a3d,Bakkt,series_a,2019-01-02 03:34:58.000000 UTC,182500000.0,15.0,"Alan Howard,CMT Digital Ventures,Chainfund Cap...","df033e28-b918-55b6-4898-c9966dd579b2,07ffde2d-...",,,...,,,,,,,,,,
2,1fc074a8-9f78-4dee-ba4e-d3a7de3096b4,Aptos,seed,2022-03-15 20:15:27.000000 UTC,200000000.0,15.0,"Andreessen Horowitz,BlockTower Capital,Coinbas...","ce91bad7-b6d8-e56e-0f45-4763c6c5ca29,2b638738-...",,,...,,,,,,,,,,
3,af0849e9-cddf-7558-1110-6b0900406945,Solugen,seed,2017-10-25 06:05:55.000000 UTC,4400000.0,15.0,"Brad Flora,Cantos,Fifty Years,FundersClub,Hack...","759ccd3b-a622-8d14-5656-86662013a1e6,0c9a00aa-...",,,...,,,,,,,,,,
4,cf6d3339-99d3-6ff2-4709-3f071d130750,Color,series_a,2015-04-21 06:59:30.000000 UTC,15000000.0,20.0,"#Angels,AME Cloud Ventures,Aaron Levie,Avichal...","65609a06-78e0-195e-7032-ff9353b82b46,7de64040-...",,,...,,,,,,,,,,


In [213]:
# we fill the new columns with the relevant values
for uuid in df_list_SuccessComp[3]['org_uuid'].unique():
    for round in invest_type_array:
        for col in relevant_col_list:
            df_list_SuccessComp[3][round + '_' + col].loc[(df_list_SuccessComp[3]['org_uuid'] == uuid)] = df_list_SuccessComp[3].loc[
                                                                                                     (df_list_SuccessComp[3]['org_uuid'] == uuid) & 
                                                                                                     (df_list_SuccessComp[3]['investment_type'] == round)].loc[:,col]
            df_list_UnsuccessComp[3][round + '_' + col].loc[(df_list_UnsuccessComp[3]['org_uuid'] == uuid)] = df_list_UnsuccessComp[3].loc[
                                                                                                     (df_list_UnsuccessComp[3]['org_uuid'] == uuid) & 
                                                                                                     (df_list_UnsuccessComp[3]['investment_type'] == round)].loc[:,col]

df_list_SuccessComp[3].head()

/var/folders/zm/n63_ssc512d4ygg3lbvwql400000gq/T/ipykernel_53439/1384574451.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list_SuccessComp[3][round + '_' + col].loc[(df_list_SuccessComp[3]['org_uuid'] == uuid)] = df_list_SuccessComp[3].loc[
/var/folders/zm/n63_ssc512d4ygg3lbvwql400000gq/T/ipykernel_53439/1384574451.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list_UnsuccessComp[3][round + '_' + col].loc[(df_list_UnsuccessComp[3]['org_uuid'] == uuid)] = df_list_UnsuccessComp[3].loc[


,org_uuid,org_name,investment_type,round_created_at,raised_amount_usd,investor_count,investors_names,investors_uuids,seed_round_created_at,seed_raised_amount_usd,...,angel_round_created_at,angel_raised_amount_usd,angel_investor_count,angel_investors_names,angel_investors_uuids,pre_seed_round_created_at,pre_seed_raised_amount_usd,pre_seed_investor_count,pre_seed_investors_names,pre_seed_investors_uuids
0,b42a3bec-1f35-42a3-9d5a-e6d3199440c6,Yuga Labs,seed,2022-03-22 22:57:00.000000 UTC,450000000.0,39.0,"Adidas,Animoca Brands,Artist Capital Managemen...","eec354de-1ed2-254d-647b-f218643791d9,8f1b5d66-...",2022-03-22 22:57:00.000000 UTC,450000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8335acfe-629e-4073-b857-205b3b5d2a3d,Bakkt,series_a,2019-01-02 03:34:58.000000 UTC,182500000.0,15.0,"Alan Howard,CMT Digital Ventures,Chainfund Cap...","df033e28-b918-55b6-4898-c9966dd579b2,07ffde2d-...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1fc074a8-9f78-4dee-ba4e-d3a7de3096b4,Aptos,seed,2022-03-15 20:15:27.000000 UTC,200000000.0,15.0,"Andreessen Horowitz,BlockTower Capital,Coinbas...","ce91bad7-b6d8-e56e-0f45-4763c6c5ca29,2b638738-...",2022-03-15 20:15:27.000000 UTC,200000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,af0849e9-cddf-7558-1110-6b0900406945,Solugen,seed,2017-10-25 06:05:55.000000 UTC,4400000.0,15.0,"Brad Flora,Cantos,Fifty Years,FundersClub,Hack...","759ccd3b-a622-8d14-5656-86662013a1e6,0c9a00aa-...",2017-10-25 06:05:55.000000 UTC,4400000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cf6d3339-99d3-6ff2-4709-3f071d130750,Color,series_a,2015-04-21 06:59:30.000000 UTC,15000000.0,20.0,"#Angels,AME Cloud Ventures,Aaron Levie,Avichal...","65609a06-78e0-195e-7032-ff9353b82b46,7de64040-...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:
# we drop the duplicate rows
print(df_list_SuccessComp[3].shape, df_list_UnsuccessComp[3].shape)

df_list_SuccessComp[3].drop_duplicates(), df_list_UnsuccessComp[3].drop_duplicates()

print(df_list_SuccessComp[3].shape, df_list_UnsuccessComp[3].shape)

(4652, 28) (10353, 28)
(4652, 28) (10353, 28)


3. Join all of the dataframes together.

In [290]:
'''Merging all of the successful data into one dataframe and creating a column called "success_flag" 
which will denote if the company is successful or not '''
df_SuccessfulCompanies_merged = reduce(lambda  left,right: pd.merge(left, right, on=['org_uuid'],
                                                                    how='outer', 
                                                                    suffixes=('', '_y')),
                                                                    df_list_SuccessComp)
df_SuccessfulCompanies_merged['success_flag']  = 1

# drop duplicate columns
df_SuccessfulCompanies_merged.drop(df_SuccessfulCompanies_merged.filter(regex='_y$').columns, axis=1, inplace=True)

'''Merging all of the unsuccessful data into one dataframe and creating a column called "success_flag" 
which will denote if the company is successful or not '''


df_UnsuccessfulCompanies_merged = reduce(lambda  left,right: pd.merge(left, right, on=['org_uuid'],
                                                                      how='outer', 
                                                                      suffixes=('', '_y')),
                                                                      df_list_UnsuccessComp)
df_UnsuccessfulCompanies_merged['success_flag']  = 0

# drop duplicate columns
df_UnsuccessfulCompanies_merged.drop(df_UnsuccessfulCompanies_merged.filter(regex='_y$').columns, axis=1, inplace=True)

'''Merge the successful and unsuccessful data into one datafram'''
df_All_Companies_merged = pd.concat([df_SuccessfulCompanies_merged, df_UnsuccessfulCompanies_merged], ignore_index=True, sort=False)

In [291]:
# sanity check
df_All_Companies_merged.head()

,org_uuid,org_name,domain,status,founded_on,category_list,category_groups_list,country_code,city,short_description,long_description,founder_linkedin_url,success_flag
0,a2323b6c-29b9-4750-905c-cdcbd9ce92b8,10X Banking,10xbanking.com,operating,2016-01-01,"Banking,Financial Services,FinTech,Software","Financial Services,Lending and Investments,Sof...",GBR,London,10X Banking is a financial service technology ...,"10x Banking (10x), a financial services techno...",https://www.linkedin.com/in/antony-jenkins/,1
1,cf3b00cf-ffbf-0e4f-6427-a60e5060de39,10X Genomics,10xgenomics.com,ipo,2012-01-01,"Biotechnology,Genetics,Health Care,Medical","Biotechnology,Health Care,Science and Engineering",USA,Pleasanton,10X Genomics ELV solutions and security system...,10x Genomics is creating revolutionary DNA seq...,https://www.linkedin.com/in/ben-hindson-09739813/,1
2,cf3b00cf-ffbf-0e4f-6427-a60e5060de39,10X Genomics,10xgenomics.com,ipo,2012-01-01,"Biotechnology,Genetics,Health Care,Medical","Biotechnology,Health Care,Science and Engineering",USA,Pleasanton,10X Genomics ELV solutions and security system...,10x Genomics is creating revolutionary DNA seq...,https://www.linkedin.com/in/serge-saxonov-7b42...,1
3,0680987f-23cd-4fa5-850c-38326fffaa36,"111, Inc.",corporate.111.com.cn,ipo,2010-01-01,"Health Care,Medical",Health Care,CHN,Shanghai,"111, Inc. operates as a digital and mobile hea...","111,Inc. is a leading digital and mobile healt...",https://www.linkedin.com/in/junling-liu-2b90724/,1
4,a79a6ae8-3c74-0db7-7e6b-b4f7662d37e8,17zuoye,ucenter.17zuoye.com,ipo,2011-01-01,"E-Learning,EdTech,Education,Internet,Language ...","Education,Internet Services,Software",CHN,Shanghai,17zuoye is an online learning platform for K-1...,17zuoye is an online learning platform for K-1...,https://www.linkedin.com/in/dun-xiao-3622629/,1


Now we save the dataframe


In [68]:
df_All_Companies_merged.to_csv('data/Data_merged_no_founder_info.csv')